In [1]:
import pandas as pd 
import torch 
import numpy as np

In [2]:
data_dir = "/Users/liv/trackML/train_100_events/"

In [3]:
from trackml.dataset import load_event

hits, cells, particles, truth = load_event(data_dir + 'event000001000')

In [4]:
#hits = hits.merge(cells, on='hit_id')
hits = hits.merge(truth, on='hit_id')
hits = hits.merge(particles, on='particle_id')

In [ ]:
hits

In [80]:
hits['r'] = np.sqrt(hits['x']**2 + hits['y']**2)
hits["pt"] = np.sqrt(hits.px**2 + hits.py**2)
hits = hits.sort_values(['r', 'z'])

In [138]:
hits[['prev_r', 'prev_z']] = hits.groupby('particle_id').shift(1)[['r', 'z']] 
hits[['prev_prev_r', 'prev_prev_z']] = hits.groupby('particle_id').shift(2)[['r', 'z']] 
hits = hits.reset_index()
hits[['next_r', 'next_z', 'next_x', 'next_y']] = hits.groupby('particle_id').shift(-1)[['r', 'z', 'x', 'y']] 
seeds = hits.groupby('particle_id').head(3)
#hits_no_seeds = hits[~hits['hit_id'].isin(seeds.hit_id)]
hits_wo_last= hits.groupby('particle_id').head(-1)

In [ ]:
from preprocessing.hit_candidates import process



In [147]:
from sklearn.neighbors import NearestNeighbors
n_comp_hits = 4

nbrs = NearestNeighbors(n_neighbors=n_comp_hits+1, algorithm='auto').fit(hits_wo_last[["next_x", "next_y", "next_z"]])  # Add 1 to exclude self
distances, indices = nbrs.kneighbors(hits_wo_last[["next_x", "next_y", "next_z"]])
correct_hit = indices[:, 0]
# Remove the first column (self) from the results
neighbor_r = hits_wo_last.iloc[indices.flatten()]['r'].values.reshape(indices.shape)
neighbor_z = hits_wo_last.iloc[indices.flatten()]['z'].values.reshape(indices.shape)
sorted_order = np.lexsort((neighbor_z, neighbor_r))
sorted_closest_indices = np.take_along_axis(indices, sorted_order, axis=1)

positions_in_sorted = np.argwhere(sorted_closest_indices == correct_hit[:, None])


In [157]:
np.unique(positions_in_sorted[:,1], return_counts=True)

(array([0, 1, 2, 3, 4]), array([14917, 20420, 22416, 19329, 15658]))

In [156]:
positions_in_sorted[:,1]

array([0, 0, 0, ..., 4, 4, 4])

In [ ]:
np.unique(array, return_counts=True)

In [ ]:
np.where(correct_hit )

In [146]:
sorted_closest_indices

array([[    0,    69,    80,   198,  2422],
       [    1,  1285,  2764,  8961,  9546],
       [    2,   258,   376,   413,  1254],
       ...,
       [89017, 90850, 92471, 92641, 92737],
       [90164, 90611, 92503, 92565, 92738],
       [88984, 90323, 90530, 92627, 92739]])

In [137]:
indices

array([[    0,  2422,    69,    80,   198,   337],
       [    1,  2764,  1285,  9546,  8961,   158],
       [    2,  1254,   413,   376,   258,  8012],
       ...,
       [92737, 92641, 92471, 90850, 89017, 92448],
       [92738, 92503, 90164, 90611, 92565, 89390],
       [92739, 90530, 92627, 88984, 90323, 92680]])

In [ ]:
sorted_closest_indices

In [134]:
hits_wo_last.next_hit_id

13479      14029.0
45123      48253.0
48029      51209.0
49751      55724.0
49331      52565.0
            ...   
99922     115556.0
100791    117832.0
99859     115450.0
99265     113461.0
99298     114048.0
Name: next_hit_id, Length: 92740, dtype: float64

In [136]:
sorted_closest_indices

array([[    0,    69,    80,   198,   337,  2422],
       [    1,   158,  1285,  2764,  8961,  9546],
       [    2,   258,   376,   413,  1254,  8012],
       ...,
       [89017, 90850, 92448, 92471, 92641, 92737],
       [89390, 90164, 90611, 92503, 92565, 92738],
       [88984, 90323, 90530, 92627, 92680, 92739]])

In [119]:
hits_no_seeds = hits_wo_next[~hits_wo_next['hit_id'].isin(seeds.hit_id)]


In [109]:
features = ["r", "z", "prev_z", "prev_r"]

selected_cols = [x for x in hits.columns if x in features or x.startswith('comp_')]
hits_no_seeds[selected_cols]

,z,r,prev_r,prev_z,comp_1_r,comp_1_z,comp_2_r,comp_2_z,comp_3_r,comp_3_z,comp_4_r,comp_4_z,comp_5_r,comp_5_z
5023,-1097.500000,30.647999,30.459892,-822.500000,30.333883,-822.500000,31.164457,-822.500000,35.312302,-957.500000,32.359543,-822.500000,31.679310,-822.500000
23113,69.967300,33.140228,32.706966,69.721603,32.652645,41.786201,32.299473,40.603901,32.562489,39.892601,71.569366,84.506699,31.498953,36.527199
18869,-0.828129,33.156097,32.645306,-0.803123,33.037563,-0.715626,33.608986,-3.640630,71.566742,-3.753120,34.203476,-1.615630,33.946068,-5.220540
20141,-0.771877,33.193848,32.675339,-0.803123,72.476318,0.096874,33.103096,-1.844010,33.273102,0.814899,33.229130,-3.471870,71.147949,3.190620
23444,71.497299,33.244324,32.725914,70.484596,32.736710,64.805702,72.216354,139.218994,71.620033,135.934006,32.585632,61.316399,32.415073,101.128998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99922,-54.400002,1021.834167,1018.811401,-48.599998,1018.811401,-48.599998,819.967773,37.799999,822.536133,-27.000000,815.357422,-97.599998,819.724304,-37.799999
100791,975.599976,1021.915405,1016.201660,975.599976,1016.201660,975.599976,820.140259,758.799988,816.484741,704.799988,820.009094,786.200012,823.840576,655.799988
99859,-54.400002,1022.018311,1018.961365,-48.599998,1018.961365,-48.599998,1016.290344,-54.400002,823.890259,-37.799999,817.166016,-76.000000,1015.986084,-97.599998
99265,-769.599976,1022.079041,1016.648682,-758.799988,1016.648682,-758.799988,820.887268,-569.400024,822.404663,-645.000000,1018.285339,-807.799988,818.612671,-758.799988
